# Prophet

[**Prophet**](https://facebook.github.io/prophet/) is a forecasting algorithm developed at Facebook and targeting "business time series" - time series derived from human activity, typically having seasonality and trend on human time scales like days, weeks, years. Features of the Prophet algorithm are:

- model is based on overall trend, seasonality, and holidays
- "analyst in the loop": ability to incorporate domain knowledge to tune parameters of the model

## Preamble

In [ ]:
import data_science_learning_paths
import forecast_lab

In [ ]:
data_science_learning_paths.setup_plot_style()

In [ ]:
import pandas
import seaborn
import matplotlib.pyplot as plt
import numpy

## Example: Forecasting Taxi Trips

In [ ]:
from prophet import Prophet

In [ ]:
taxi_trips = data_science_learning_paths.datasets.read_chicago_taxi_trips_daily()

In [ ]:
taxi_trips.head()

Prophet expects the input time series in the format of a `pandas.DataFrame` with columns `ds` for time and `y` for values.

In [ ]:
taxi_trips_p  = pandas.DataFrame(taxi_trips).reset_index()
taxi_trips_p.columns = ["ds", "y"]

In [ ]:
taxi_trips_p.head()

In [ ]:
taxi_trips_p["ds"].max()

Now the model can be fitted to the time series:

In [ ]:
%time prophet_model = Prophet().fit(taxi_trips_p)

In order to produce a forecast, one needs to call the model's helper method `make_future_dataframe` with the parameters of the desired forecast:

In [ ]:
fd = prophet_model.make_future_dataframe(periods=10, freq="D")
fd[fd["ds"] > "2017-05-31"]

In [ ]:
ts_forecast = prophet_model.predict(
    prophet_model.make_future_dataframe(periods=100, freq="D")
)

The forecast result is a dataframe with values for the historical time series plus the future timeframe:

In [ ]:
ts_forecast[ts_forecast["ds"] > taxi_trips.index.max()]["yhat"].shape

In [ ]:
ts_forecast.tail()

In [ ]:
prophet_model.plot(ts_forecast[ts_forecast["ds"] > taxi_trips.index.max()], );

We can get some insights into the model by plotting its components - an overall trend as well as seasonal components:

In [ ]:
prophet_model.plot_components(ts_forecast);


We provide a wrapper class to use a Prophet model in the `ForecastEvaluation`:

In [ ]:
forecast_lab.ProphetWrapper??

## Evaluation

In [ ]:
forecast_lab.ForecastEvaluation(
        ts=taxi_trips["Trips"],
        forecasting=forecast_lab.ProphetWrapper(freq="d"),
        test_window_size=120,
        train_window_size=365,
        metrics={
            "MAPE": data_science_learning_paths.mlp.mean_absolute_percentage_error,
            "RSME": data_science_learning_paths.mlp.root_mean_squared_error
        }
).evaluate(
    k=10, 
    plot_segments=True,
    plot_residuals=True,
).get_metrics().mean()

## Adding External Variables

Among other configuration options, Prophet is built with the option to add additional features (e.g., holiday calendars) in mind. Refer to [this section of the Prophet documentation](https://facebook.github.io/prophet/docs/seasonality,_holiday_effects,_and_regressors.html#additional-regressors) for more information.

**Exercise: Adding Holidays to the Prophet Model**

Try adding a US holiday calendar to the taxi trips forecast model using Prophet. Then implement this for the `ProphetWrapper` using the `ext_vars` parameter.

## Summary

**Pros**

+ a model made for business time series with typical seasonal patterns
+ default hyperparameters often already produce an acceptable forecast
+ incorporating external variables is supported

**Cons**

- quality of the library documentation is lacking

## References

- [Paper: Forecasting at scale](https://peerj.com/preprints/3190/)

---
_This notebook is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/). Copyright © 2018-2026 [Point 8 GmbH](https://point-8.de)_